<a href="https://colab.research.google.com/github/buganart/descriptor-transformer/blob/main/train_notebook/melgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

In [ ]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

# Description

This notebook is used for training melgan and log results to the wandb project "demiurge/melgan-neurips". The [buganart/melgan-neurips](https://github.com/buganart/melgan-neurips) code is based on the [descriptinc/melgan-neurips repository](https://github.com/descriptinc/melgan-neurips).

To start training the melgan, user will need to specify **audio_db_dir** to locate a music folder in the mounted Google Drive. All the data in the folder will be used for training and evaluating the model traininig process. **experiment_dir** and **melgan_output_dir** are the path where the data generated from the melgan training process is saved. 

In case the run is stopped, and the user want to resume such run, please specify wandb run id in the **resume_run_id**. For all the training arguments, please see [descriptinc/melgan-neurips repository](https://github.com/descriptinc/melgan-neurips).

In [ ]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
# audio_db_dir = "/content/drive/My Drive/AUDIO DATABASE/RAW Sessions/Roberto Studio Material" #@param {type:"string"}
audio_db_dir = "/content/drive/My Drive/AUDIO DATABASE/TESTING" #@param {type:"string"}
experiment_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan" #@param {type:"string"}
melgan_output_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-violingan/melgan-outputs" #@param {type:"string"}

#@markdown ### Resumption of previous runs
#@markdown Optional resumption arguments below, leaving both empty will start a new run from scratch. 
#@markdown - The ID can be found on wandb. 
#@markdown - It's 8 characters long and may contain a-z letters and digits (for example `1t212ycn`).

#@markdown Resume a previous run 
resume_run_id = "" #@param {type:"string"}
#@markdown Load initial weights from a previous run to start a new run.
load_from_run_id = "" #@param {type:"string"}

#@markdown train argument
n_mel_channels = 80 #@param {type: "integer"}
ngf = 32 #@param {type: "integer"}
n_residual_layers = 3 #@param {type: "integer"}

ndf = 16 #@param {type: "integer"}
num_D = 3 #@param {type: "integer"}
n_layers_D = 4 #@param {type: "integer"}
downsamp_factor = 4 #@param {type: "integer"}
#@markdown - ratios should be list in string format, with product of elements = 256?
ratios = "[8,8,2,2]" #@param {type: "string"}

lambda_feat = 10 #@param {type: "integer"}
#cond_disc: action="store_true"
learning_rate = 1e-4 #@param {type: "number"}
pad_mode = "reflect" #@param ["reflect", "replicate"]

batch_size = 16 #@param {type: "integer"}
seq_len = 8192 #@param {type: "integer"}
sampling_rate = 44100 #@param {type: "integer"}

epochs = 3000 #@param {type: "integer"}
log_interval = 100 #@param {type: "integer"}
save_interval = 1000 #@param {type: "integer"}
n_test_samples = 8 #@param {type: "integer"}

notes = "" #@param {type: "string"}

import re
from pathlib import Path

audio_db_dir = Path(audio_db_dir)
melgan_output_dir = Path(melgan_output_dir)
experiment_dir = Path(experiment_dir)

#check ratios
import numpy as np
ratios_str = ratios
ratios_str = ratios_str.replace(" ", "")
ratios_str = ratios_str.strip("][").split(",")
ratios_str = [int(i) for i in ratios_str]
ratios_str = np.array(ratios_str)


for path in [experiment_dir, melgan_output_dir]:
    path.mkdir(parents=True, exist_ok=True)

if not audio_db_dir.exists():
    raise RuntimeError(f"audio_db_dir {audio_db_dir} does not exists.")

if resume_run_id and load_from_run_id:
    raise RuntimeError("Only set `resume_run_id` or `load_from_run_id`.")

def check_wandb_id(run_id):
    if run_id and not re.match(r"^[\da-z]{8}$", run_id):
        raise RuntimeError(
            "Run ID needs to be 8 characters long and contain only letters a-z and digits.\n"
            f"Got \"{run_id}\""
        )

check_wandb_id(resume_run_id)
check_wandb_id(load_from_run_id)

In [ ]:
#@title Clone melgan repo

!git clone https://github.com/buganart/melgan-neurips

In [ ]:
#@title Install Dependencies

%cd /content/melgan-neurips
%pip install -r requirements.txt
clear_on_success("Dependencies installed.")

In [ ]:
#@title Copy audio files to runtime

local_wav_dir = Path("/content/wavs/")
!find "{audio_db_dir}" -maxdepth 1 -type f | xargs -t -d "\n" -I'%%' -P 10 -n 1 rsync -a '%%' "$local_wav_dir"/
clear_on_success("All files copied to this runtime.")


In [ ]:
#@title Split train/test dataset

# os.environ["WANDB_MODE"] = "dryrun"
!python split_dataset.py --data_path "$local_wav_dir"

print("TRAIN FILES")
!head -n3 train_files.txt
print('...')
!tail -n3 train_files.txt

print()
print("TEST FILES")
!head -n3 test_files.txt
print('...')
!tail -n3 test_files.txt

In [ ]:
#@title TRAIN

# This done a bit weirdly because setting PYTHONPATH=$PWD removes variables afterwards. A colab bug, maybe.
!env PYTHONPATH="$(pwd)" python scripts/train.py \
--save_path "$experiment_dir" \
--data_path . \
--resume_run_id "$resume_run_id" \
--load_from_run_id "$load_from_run_id" \
--n_mel_channels "$n_mel_channels" \
--ngf "$ngf" \
--n_residual_layers "$n_residual_layers" \
--ndf "$ndf" \
--num_D "$num_D" \
--n_layers_D "$n_layers_D" \
--downsamp_factor "$downsamp_factor" \
--ratios "$ratios" \
--lambda_feat "$lambda_feat" \
--learning_rate "$learning_rate" \
--pad_mode "$pad_mode" \
--batch_size "$batch_size" \
--seq_len "$seq_len" \
--sampling_rate "$sampling_rate" \
--epochs "$epochs" \
--log_interval "$log_interval" \
--save_interval "$save_interval" \
--n_test_samples "$n_test_samples" \
--notes "$notes"